**Imports**

In [ ]:
# imports
from selenium import webdriver
import csv
import time
import codecs
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from csv import DictWriter
import os

driver = webdriver.Chrome('/Users/joshua/Documents/chromedriver')

**Accessing LinkedIn Account**

In [ ]:
# defining new variable passing two parameters

# driver.get method() will navigate to a page given by the URL address
driver.get('https://www.linkedin.com')

# locate email form by_class_name
username = driver.find_element("name", "session_key")

# send_keys() to simulate key strokes
username.send_keys('') #email-address

# sleep for 0.5 seconds
time.sleep(0.5)

# locate password form by_class_name
password = driver.find_element("name", "session_password")

# send_keys() to simulate key strokes
password.send_keys('')#password
time.sleep(0.5)
                         
from selenium.webdriver.common.by import By
# locate submit button by_xpath
log_in_button = driver.find_element(By.CLASS_NAME,"sign-in-form__submit-button")

# .click() to mimic button click
log_in_button.click()

**Getting The URLs In Google**

In [ ]:
driver.get('https:www.google.com')
time.sleep(3)

search_query = driver.find_element('name', 'q')
search_query.send_keys('site:linkedin.com/in/ AND "University of Santo Tomas"')

search_query.send_keys(Keys.RETURN)

driver.find_element(By.XPATH, '//*[@id="hdtb-msb"]/div[1]/div/div[2]/a').click()
linkedin_urls = []

for i in range(14):
    # locate URL by_class_name
    linkedin = [my_elem.get_attribute("href") for my_elem in driver.find_elements(By.CSS_SELECTOR,  "a.WlydOe")]

    from parsel import Selector

    driver.page_source

    linkedin = [url for url in linkedin]
    linkedin_urls = linkedin_urls + linkedin
    
    time.sleep(5)
    
    driver.find_element(By.XPATH,'//*[@id="pnnext"]/span[2]').click()

**Getting The Data On LinkedIn**

In [ ]:
import requests

# function to ensure all key data fields have a value
def validate_field(field):# if field is present pass if field:pass
# if field is not present print text else:
    field = 'No results'
    return field

# For loop to iterate over each URL in the list
for linkedin_url in linkedin_urls:

    # get the profile URL 
    driver.get(linkedin_url)

    # add a 5 second pause loading each URL
    time.sleep(5)

    # assigning the source code for the webpage to variable sel
    sel = Selector(text=driver.page_source)
    

    src = driver.page_source
 
    soup = BeautifulSoup(src, 'lxml')
    
    details = soup.find('div', {'class': 'mt2 relative'})
    
    name_loc = details.find('h1')
    name = name_loc.get_text().strip()

    job_title_loc = details.find('div',{'class': 'text-body-medium break-words'})
    job_title = job_title_loc.get_text().strip()

    location_loc = details.find('span', {'class': 'text-body-small inline t-black--light break-words'})
    location = location_loc.get_text().strip()

    company_college = details.find_all('div', {'class': 'inline-show-more-text'})

    if len(company_college) == 2:
        company_loc = company_college[0]
        college_loc = company_college[1]
        company = company_loc.get_text().strip()
        college = college_loc.get_text().strip()
    else:
        college_loc = company_college[0]
        college = college_loc.get_text().strip()
        company = "No results"
    
    with open("Linkedin_Data.csv",'a', newline='') as f:
        writer = DictWriter(f, fieldnames = ['Name', 'Job Title', 'Company', 'College', 'Location', 'URL'])
    
        if os.stat("Linkedin_Data.csv").st_size == 0:
            DictWriter.writeheader(writer)

        writer.writerow({'Name': name, 'Job Title': job_title, 'Company': company, 'College': college, 'Location': location, 'URL': linkedin_url})